In [7]:
import os
from langchain.llms import HuggingFaceTextGenInference, OpenAI
from dotenv import load_dotenv
load_dotenv()

True

Wir definieren nun zwei Hilfsfunktionen um einfach zwischen GPT4 und unserem lokalen Sprachmodell zu wechseln.

In [3]:
def getLLM_openai(): #openai API key kommt aus der .env Datei
    #return ChatOpenAI(model="gpt-4-turbo-preview")
    return OpenAI(request_timeout=10, model_name="gpt-4-turbo-preview") #10s timeout, model_name = "gpt-3.5-turbo"


def getLLM_TGI(): 
     llm = HuggingFaceTextGenInference(inference_server_url=os.environ['TGI_URL'], max_new_tokens=2048, top_k=10, top_p=0.95, typical_p=0.95, temperature=0.01, repetition_penalty=1.03) 
     return llm
 
 
llm = getLLM_openai()

/opt/anaconda3/envs/ml/lib/python3.9/site-packages/langchain_community/llms/openai.py:249: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain_community.chat_models import ChatOpenAI`
  warnings.warn(
/opt/anaconda3/envs/ml/lib/python3.9/site-packages/langchain_community/llms/openai.py:1070: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain_community.chat_models import ChatOpenAI`
  warnings.warn(


# Erste Experimente
In der nächsten Zelle rufen wir das LLM einfach auf.

In [4]:
res = llm.invoke("Warum ist der Himmel blau? Antworte in drei Sätzen.")
print(res)

Retrying langchain_community.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised Timeout: Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=10).


Der Himmel erscheint blau, weil die Moleküle in der Erdatmosphäre das Sonnenlicht streuen, wobei das blaue Licht stärker gestreut wird als das Licht anderer Farben. Dieses Phänomen, bekannt als Rayleigh-Streuung, tritt auf, weil blaues Licht kürzere Wellenlängen hat und somit effizienter in alle Richtungen gestreut wird. Daher dominiert das blaue Licht im gesamten Himmel, was uns der Himmel tagsüber in seiner charakteristischen Farbe erscheint.


# RAG workflow
Im folgenden stellen wir einen Beispielhaften RAG Workflow dar, der Daten von der Wikipedia Seite von Tibet als Grundlage nimmt.

In [5]:
import prompts
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, GPT4AllEmbeddings, HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA

loader = WebBaseLoader("https://de.wikipedia.org/wiki/Tibet")
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 100)
splits = text_splitter.split_documents(loader.load())

Wir definieren uns nun ein Embedding für die deutsche Sprache und nehmen Chroma als Vektordatenbank. Diese benötigen wir für die Suche nach passenden Dokumenten für unsere Anfrage an das Sprachmodell

In [6]:
emb = HuggingFaceEmbeddings(model_name='aari1995/German_Semantic_STS_V2')
vectorstore = Chroma.from_documents(documents=splits,embedding=emb, persist_directory="./chroma_db")
retriever = vectorstore.as_retriever()
rag_prompt = prompts.german_sauerkrautlm()

No sentence-transformers model found with name aari1995/German_Semantic_STS_V2. Creating a new one with MEAN pooling.


In der folgenden Zelle verketten wir die Anfrage an die Vektordatenbank mit dem Aufruf von unserem Sprachmodell in einem `rag_chain`.

In [7]:
# configure rag chain
chain_type_kwargs = {"prompt": rag_prompt}
rag_chain = RetrievalQA.from_chain_type(llm=llm, chain_type='stuff', retriever=retriever, 
                                    chain_type_kwargs=chain_type_kwargs,
                                    return_source_documents=True,
                                    )

Nun erfolgt die eigentliche Anfrage an das LLM:

In [8]:
res = rag_chain.invoke("Was weißt du zu Tibet?")
print(res['result'])

Retrying langchain_community.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised Timeout: Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=10).
Retrying langchain_community.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised Timeout: Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=10).
Retrying langchain_community.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised Timeout: Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=10).
Retrying langchain_community.llms.openai.completion_with_retry.<locals>._completion_with_retry in 8.0 seconds as it raised Timeout: Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=10).
Retrying langchain_community.llms.openai

Tibet ist eine Region in Zentralasien, welche kulturell und historisch von tibetisch-sprachigen Völkern geprägt wurde. Sie liegt abgeschieden südlich des Himalaya-Gebirges und entwickelte schon vor dem 7. Jahrhundert eigenständige Staaten wie Shangshung und Tubo. Tibet geriet im 13. Jahrhundert kurzzeitig unter mongolische Herrschaft und hatte bis ins 20. Jahrhundert hinein ein eigenes Staatswesen. Derzeit gehört Tibet zur Volksrepublik China, jedoch ist diese Zugehörigkeit völkerrechtlich umstritten. Seit 1959 existiert eine tibetische Exilregierung, die international nicht offiziell anerkannt, aber von vielen Ländern unterstützt wird.
